<a href="https://colab.research.google.com/github/k-ferry/cs676-fall-2025/blob/main/project-1/deliverable1/deliverable1v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credibility Scoring for eBay Soccer Card Listings in a RAG Chatbot

# Abstract
We propose and evaluate a credibility scoring component for a Retrieval-Augmented Generation (RAG) chatbot that surfaces soccer card listings from eBay.  The system assigns a per-URL credibility score to help users judge the reliability of cited sources and prioritize higher-quality listing.  Our approach is hybrid: a transparent rule-based layer (domain/transport, content heuristics, and eBay-specific seller/listing signals) plus a lighweight ML calibration (logistic regression) that learns to reweight signal from labeled data.  We present the prototype algorithm, a stable JSON API, initial experiment and roadmap to production.

# Introduction & Scope
Large language models augemented with retrieval (RAG) can cite live source for pricing, comps, and context.  However, cited sources vary in reliability.  For sports cards on marketplaces like eBay, credibility as the likelihood that a liting accurately respresents what it claims (authenticity, condition clarity, seller reliability, sufficient evidence) and that it's a solid source to support chatbot responses.

The report describes the design and evaluation of a credibility scorer that:


*   Takes a URL as input
*   Computes interpretable signals (domain prior seller feedback %, returns policy, specific/keywords, etc)
*   Aggregates them into a 0-100 score, optionally calibrated via a small supervised model
*   Returns a stable JSON the chatbot can display alongside citations.





# 2. Problem Framing
Goal: Assign each listing a credibility score useful for ranking and for user-facing explanation.

Assumption:


*   Seller reputation (feedback %, count, Top Rated) correlates with trust.
*   Listings with clear photos, specific card attributes (grade, serial, set), and return policies are more reliable
*   Very short or ambiguous descriptions reduce credibility
*   Sentiment terms can be a weak prior (e.g. "creased", "OC", "gem", "clean")

Next-steps:  We do not yet authenticate PSA certs, verify images, or compute fair market value.  These are roadmap items.

In [ ]:
# Imports & Config

# Purpose of this cell:
# - Import only what we need for the core prototype to keep fast and stable
# - Set conservaitve network defaults (timeouts/headers) so production integration goes smoother.
# - Pre-compile a domain regex for eBay to enable platform-specific signals later.

from __future__ import annotations
# ^ Enables "forward references" for type hints (e.g., using a class/type name before it's defined)
# Helps keep the file order flexible

import dataclasses
import json
import math
import re
import time
import typing as t
from dataclasses import dataclass
from datetime import datetime, timezone
from urllib.parse import urlparse
# ^ Standard-library imports only so far:
# - cataclasses/dataclass: simple typed containers for signals and score results
# - json: for serializing output/dicts
# - math: numeric helpers (Log10, exp for squashing function)
# - re: regular expressions for parsing text/html heuristics
# - time: peformance timing (ms) for basic benchmarking/metadata
# - typing: type hints, e.g., t.Sequence[float]
# - datetime/timezone: timestamps in UTC (stable logging and reproducability)
# - urllib.parse.urlparse: strict URL validation and parsing

# Optional imports guarded at runtime; keep import-time light and notebook robust.
try:
    import requests  # HTTP client used only when dry_run=False
except Exception:
    requests = None  # # If requests isn't available (or blocked), we degrade gracefully.
# ^ Rationale: Deliverable 1 must run even without network access (e.g., in Colab or CI).
#   The code uses a "dry_run" mode that synthesizes HTML; this keeps tests deterministic.

# Optional libs used later (pandas for tables; sklearn for D2 calibration)
try:
    import pandas as pd  # tabular display and simple data wrangling for ranked outputs
except Exception:
    pd = None # Allow notebook to run even if pandas isn't installed.

try:
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import train_test_split
    import numpy as np
except Exception:
    LogisticRegression = None
    Pipeline = None
    StandardScaler = None
    train_test_split = None
    np = None

# --- Network defaults (used only if dry_run=False) ---
DEFAULT_TIMEOUT_S = 6.0
# ^ Per-request timeout (seconds).  Conservative to avoid hanging UI calls.

DEFAULT_HEADERS = {
    # Custom UA string: polite + traceable in server logs; helps avoid some bot blocks.
    "User-Agent": "CredScorer/0.1 (+https://example.edu/project)",
    # Accept header: prefer HTML/XML; still accept anything to be resilient.
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

# --- Platform detector (precompiled regex) ---
# Matches eBay primary domain and common regional TLDs (ebay.com, ebay.co.uk, ebay.de, etc.).
EbayLike = re.compile(r"(^|\.)ebay\.(com|co\.[a-z]{2}|[a-z]{2})$", re.I)


In [ ]:
# Cell 2 - Data Structures & Stable Response Schema
# -------------------------------------------------
# Purpose
# 1) Define small, typed containers for signals and results (using dataclasses).
# 2) Provide a *single stable* JSON contract via 'response_json(...)' that every caller can rely on
#    This avoids brittle “if error, shape is different” problems when we integrate (Deliverable 3)

@dataclass
class Signal:
"""
    An interpretable scoring *component* (feature) contributing to credibility.

    Fields
    ------
    name : str
        Short identifier (e.g., "domain_prior", "seller_feedback_pct").
    value : float
        Normalized feature value in [0, 1]. Higher is better (more credible/desirable).
    weight : float
        Weight in [0, 1] reflecting this signal's *influence* in the current heuristic model.
        (In Deliverable 2, a learned model can reweight via calibration.)
    rationale : str
        Human-readable explanation so the UI/report can show *why* a score moved.
    """
    name: str
    value: float   # 0..1
    weight: float  # 0..1
    rationale: str

    def contribution(self) -> float:
              """
        Weighted contribution for this signal.
        We keep the math simple and transparent: contribution = value * weight.

        Returns
        -------
        float
            Contribution in [0, 1] (typically small), later aggregated and squashed to [0, 100].
        """

        return self.value * self.weight
@dataclass
class ScoreResult:
"""
    Internal container for an entire scoring run on a single URL.
    We keep this separate from the external JSON payload so we can evolve internals
    without breaking the API contract.

    Fields
    ------
    url : str
        The URL that was evaluated.
    status : str
        "ok" for success; or "invalid_url", "fetch_error", etc. for graceful failure modes.
    score_abs : float
        Absolute score after aggregation and squashing to [0, 100].
    score_pct : float | None
        Optional percentile vs a cohort (if provided by batch ranking).
    signals : list[Signal]
        All computed signals with values/weights and rationales.
    errors : list[str]
        Non-fatal errors/warnings encountered (e.g., network timeouts).
    meta : dict[str, t.Any]
        Extra metadata (host, is_ebay flag, timestamps, timings, version tag).
    """
    url: str
    status: str
    score_abs: float
    score_pct: float | None
    signals: list[Signal]
    errors: list[str]
    meta: dict[str, t.Any]

def response_json(result: ScoreResult) -> dict:
    """
    Convert internal ScoreResult into a *stable* JSON/dict that clients can consume.
    This is the public contract for Deliverable 1/2/3. Keep this structure stable.

    Returns
    -------
    dict
        {
          "url": <str>,
          "status": <"ok" | "invalid_url" | "fetch_error" | ...>,
          "score": {"absolute": <float 0..100>, "percentile": <float|None>},
          "signals": [{"name":..., "value":..., "weight":..., "rationale":...}, ...],
          "errors": [<str>, ...],
          "meta": { "host":..., "is_ebay":..., "fetched_at":..., "elapsed_ms":..., "fetch_ms":..., "version":"d1-0.1" }
        }
    """
    return {
        "url": result.url,
        "status": result.status,
        "score": {"absolute": result.score_abs, "percentile": result.score_pct},
        "signals": [dataclasses.asdict(s) for s in result.signals],
        "errors": result.errors,
        "meta": result.meta,
    }


In [ ]:
# Cell 3 - Utilities
# -------------------
# Purpose: small helpers that are used everywhere else
# Keep them fast, deterministic, and side-effect free



def _cheap_text(html: str) -> str:
  """

  Ultra-Fast HTML > text stripper for heuristics
  Avoid BeautifulSoup to keep dependencies light and speed high.

  Steps:
    1) Remove <script> and <style> blocks (noise).
    2) Remove all remaining HTML tags.
    3) Collapse whitespace to single spaces.
  Returns a plain text string suitable for regex/keyword scans.
  """
    text = re.sub(r"<script[\s\S]*?</script>", " ", html, flags=re.I)
    text = re.sub(r"<style[\s\S]*?</style>", " ", text, flags=re.I)
    text = re.sub(r"<[^>]+>", " ", text)          # strip tags
    text = re.sub(r"\s+", " ", text)              # normalize whitespace
    return text.strip()

def _squash_0_100(raw: float) -> float:
    """
    Map an unbound-ish raw sum of contributions to a user-friendly 0..100 score
    using a smooth logistic curve.  Center around ~0.8 (moderate sum), so small
    changes near the center are visible but extremes saturate.

    raw: float, typically ~0..1.4 (sum of value*weight across signals)
    """
    # Smooth logistic squashing to [0,100], centered around ~0.8
    x = raw - 0.8
    sig = 1 / (1 + math.exp(-3.5 * x))
    return round(100 * sig, 2)

def _percentile(x: float, arr: list[float]) -> float:
    """
    Inclusive rank percentile in 0..100.  If arr is empty, returns NaN.
    We use a simple <= rank so ties are included.
    """
    if not arr:
        return float("nan")
    rank = sum(1 for a in arr if a <= x)
    return round(100 * rank / len(arr), 2)

def _now_iso() -> str:
    """UTC timestamp in ISO-8601 for reproducible logs/metadata."""
    return datetime.now(timezone.utc).isoformat()

def _elapsed_ms(t0: float) -> int:
    """Milliseconds elapsed since montonic start to (from time.perf_counter())."""
    return int((time.perf_counter() - t0) * 1000)

def _synthetic_page_for(host: str) -> str:
    """Deterministic synthetic HTML used by dry_run=True.
    We emit and eBay-like snippet for eBay hosts, and a generic article otherwise."""
    if EbayLike.search(host or ""):
        return (
            "<html><head><title>eBay Listing</title></head><body>"
            "Top Rated Seller (99.7% positive feedback) (12450) feedback. "
            "2024 Topps Chrome UEFA Refractor PSA 10 Rookie /99 auto. "
            "Ships from New York. 30 day returns. "
            "<img/><img/><img/><img/><img/><img/>"
            "</body></html>"
        )
    return (
        "<html><body>By John Doe. Published 2023. "
        "References: https://doi.org/10.x/y "
        "This is a sample article body with some length and structure."
        "</body></html>"
    )


In [ ]:
# Cell 4 - Generic, platform-agnostic signals
# -------------------------------------------------------
# These signals work for *any* URL and provide a transparent baseline.

from typing import List
from dataclasses import dataclass

# Reuse the Signal class from Cell 2
# from your_module import Signal  # (already defined in notebook)

def _signal_domain_baseline(host: str) -> Signal:
    """
    Small, interpretable prior based on the host's TLD/category.
    Rationale: Certain domains (gov/edu) tend to have stronger editorial standards,
    while eBay is a known marketplace with established buyer protections.
    """
    h = (host or "").lower()
    if h.endswith(".gov") or h.endswith(".edu"):
        return Signal("domain_prior", 0.90, 0.10, "Academic/Gov domain prior")
    if h.endswith(".org"):
        return Signal("domain_prior", 0.75, 0.08, ".org prior (neutral-to-positive)")
    # Note: eBay handled again in eBay-specific signals, but we include a modest prior here.
    from re import compile, I
    if compile(r"(^|\.)ebay\.(com|co\.[a-z]{2}|[a-z]{2})$", I).search(h):
        return Signal("domain_prior", 0.70, 0.10, "Trusted marketplace host (eBay)")
    if h.endswith(".com"):
        return Signal("domain_prior", 0.60, 0.07, ".com baseline prior")
    return Signal("domain_prior", 0.45, 0.05, "Unknown/low-signal domain")

def _signal_transport_security(scheme: str) -> Signal:
    """
    HTTPS is a soft trust signal (tamper-resistance in transit, modern hosting).
    We don't over-weight it; just nudge up if https, down if http.
    """
    return Signal(
        "https",
        1.0 if scheme == "https" else 0.4,
        0.04,
        "HTTPS vs HTTP transport",
    )

def _signals_content_quality(html: str) -> List[Signal]:
    """
    Very lightweight text-based heuristics:
      - Content length band (too short is suspect; too long is noisy).
      - Outbound links/citations density (hints at sourcing).
      - Author/date phrase hint (weak proxy for structured content).
    These are intentionally conservative and interpretable.
    """
    s: List[Signal] = []
    text = _cheap_text(html)
    n = len(text.split())

    # Length band
    if n <= 30:
        v, why = 0.20, "Very short body"
    elif n <= 120:
        v, why = 0.55, "Short body"
    elif n <= 2500:
        v, why = 0.80, "Reasonable body length"
    else:
        v, why = 0.60, "Very long body (diminishing returns)"
    s.append(Signal("content_length", v, 0.07, why))

    # Outbound refs/links density
    cites = len(re.findall(r"(doi\.org/|https?://)\S+", text))
    s.append(Signal("citations_links", min(cites / 5, 1.0), 0.04, "Outbound refs/links density"))

    # Author/date hint
    has_authorish = bool(re.search(r"\bby\s+[A-Z][a-z]+", text))
    s.append(Signal("author_block_hint", 1.0 if has_authorish else 0.5, 0.03, "Author/date block hints"))
    return s


In [ ]:
# Cell 5 -

# Hobby lexicons (expand later as needed)
CARD_TERMS = {
    # Rookie cues & desirability
    "rookie": 0.12, "rc": 0.08,
    # Grading
    "psa 10": 0.16, "bgs 9.5": 0.10, "sgc 10": 0.08, "gem mint": 0.12,
    # Auto/serial
    "auto": 0.12, "autograph": 0.12, "/": 0.10,  # /10, /25...
    # Sets/variants
    "refractor": 0.08, "color match": 0.10, "prizm": 0.08,
    "topps": 0.06, "merlin": 0.06, "select": 0.06, "optic": 0.06,
}

_POS = {
    "grail","pc","beautiful","clean","crisp","gem","iconic",
    "undervalued","deal","bargain","goat","legend","heat",
}
_NEG = {
    "creased","damage","ding","scratches","scratched","off-center",
    "offcenter","trimmed","fake","reprint","altered","stain",
    "worst","overpriced",
}

def sentiment_features(text: str) -> list[Signal]:
    tokens = re.findall(r"[a-zA-Z\-]+", text.lower())
    pos_hits = sum(1 for w in tokens if w in _POS)
    neg_hits = sum(1 for w in tokens if w in _NEG)
    total = max(pos_hits + neg_hits, 1)
    polarity = (pos_hits - neg_hits) / total  # [-1,1]
    val = (polarity + 1) / 2                  # [0,1]
    return [Signal("sentiment", val, 0.05, f"lexicon polarity {polarity:.2f}")]

def _signals_ebay_listing(html: str) -> list[Signal]:
    s: list[Signal] = []
    text = _cheap_text(html)
    lower = text.lower()

    # Sentiment (modest weight, interpretable)
    s.extend(sentiment_features(text))

    # Seller feedback %
    m = re.search(r"(\d{1,3}\.\d)\%\s*positive feedback", text, re.I)
    if m:
        pct = float(m.group(1))
        v = 0.2 + 0.8*(pct/100.0)
        s.append(Signal("seller_feedback_pct", min(v,1.0), 0.12, f"Seller feedback {pct}%"))
    else:
        s.append(Signal("seller_feedback_pct", 0.55, 0.06, "Feedback % not found"))

    # Feedback count (log scale)
    m2 = re.search(r"\((\d{2,6})\)\s*feedback", text, re.I)
    if m2:
        cnt = int(m2.group(1))
        v = min(math.log10(max(cnt,1))/5.0 + 0.4, 1.0)
        s.append(Signal("seller_feedback_count", v, 0.08, f"Feedback count {cnt}"))

    # Top Rated Seller
    if re.search(r"top rated seller", text, re.I):
        s.append(Signal("top_rated", 1.0, 0.06, "Top Rated Seller badge"))

    # Return policy
    if re.search(r"\b(30|60)\s*day returns?\b", text, re.I):
        s.append(Signal("returns_policy", 0.92, 0.05, "30/60-day returns"))
    elif re.search(r"no returns", text, re.I):
        s.append(Signal("returns_policy", 0.50, 0.05, "No returns"))

    # Specificity score (keywords + jersey number hint)
    term_score = 0.0
    for k, w in CARD_TERMS.items():
        if k in lower:
            term_score += w
    if re.search(r"\b#?\d{1,2}\b", lower):
        term_score += 0.04
    term_score = min(term_score, 1.0)
    s.append(Signal("card_specificity_terms", term_score, 0.14, "Hobby keywords present"))

    # Year + Set
    any_year = bool(re.search(r"\b(19|20)\d{2}\b", text))
    any_set  = bool(re.search(r"(prizm|topps|merlin|select|optic|megacracks|chrome)", lower))
    s.append(Signal("year_set_hint", 1.0 if (any_year and any_set) else 0.6, 0.06, "Year+Set mentioned"))

    # Image count (thoroughness proxy)
    imgs = len(re.findall(r"<img\b", html, re.I))
    if imgs >= 8:
        s.append(Signal("image_count", 0.95, 0.05, f"{imgs} images"))
    elif imgs >= 4:
        s.append(Signal("image_count", 0.75, 0.05, f"{imgs} images"))
    else:
        s.append(Signal("image_count", 0.55, 0.05, f"{imgs} images"))

    # Shipping hint
    if re.search(r"ships from\s+[A-Za-z ]+", lower):
        s.append(Signal("shipping_from", 0.70, 0.03, "Ships-from present"))

    return s


In [ ]:
# Cell 6 -

def score_url(
    url: str,
    *,
    dry_run: bool = False,
    cohort_scores: t.Sequence[float] | None = None,
    session: t.Any | None = None,
) -> dict:
    """Evaluate a reference URL and return structured credibility JSON."""
    t0 = time.perf_counter()
    errors: list[str] = []
    signals: list[Signal] = []

    # 1) URL validation
    try:
        parsed = urlparse(url)
        if parsed.scheme not in {"http", "https"} or not parsed.netloc:
            raise ValueError("URL must include http(s) scheme and host")
        host = parsed.hostname or ""
    except Exception as e:
        result = ScoreResult(
            url=url, status="invalid_url",
            score_abs=0.0, score_pct=None, signals=[],
            errors=[f"invalid_url: {e}"],
            meta={"fetched_at": _now_iso(), "elapsed_ms": _elapsed_ms(t0)},
        )
        return response_json(result)

    # 2) Domain/transport priors
    signals.append(_signal_domain_baseline(host))
    signals.append(_signal_transport_security(parsed.scheme))

    # 3) Fetch or synthesize
    html: str | None = None
    status: str = "ok"
    fetched_ms = None
    if dry_run:
        html = _synthetic_page_for(host)
        fetched_ms = _elapsed_ms(t0)
    else:
        if requests is None:
            errors.append("requests_not_available")
            status = "fetch_error"
        else:
            try:
                sess = session or requests.Session()
                r = sess.get(url, headers=DEFAULT_HEADERS, timeout=DEFAULT_TIMEOUT_S)
                fetched_ms = _elapsed_ms(t0)
                if r.status_code >= 400:
                    raise RuntimeError(f"HTTP {r.status_code}")
                html = r.text
            except Exception as e:
                errors.append(f"fetch_error: {e}")
                status = "fetch_error"

    # 4) Content & platform signals
    if html:
        try:
            signals.extend(_signals_content_quality(html))
        except Exception as e:
            errors.append(f"content_parse_error: {e}")
        try:
            if EbayLike.search(host or ""):
                signals.extend(_signals_ebay_listing(html))
        except Exception as e:
            errors.append(f"ebay_parse_error: {e}")

    # 5) Aggregate
    raw = sum(s.contribution() for s in signals)
    abs_score = _squash_0_100(raw)

    # 6) Optional percentile vs cohort
    pct = None
    if cohort_scores:
        try:
            pct = _percentile(abs_score, list(cohort_scores))
        except Exception as e:
            errors.append(f"percentile_error: {e}")

    # 7) Unified JSON
    result = ScoreResult(
        url=url, status=status,
        score_abs=abs_score, score_pct=pct,
        signals=signals, errors=errors,
        meta={
            "host": host,
            "is_ebay": bool(EbayLike.search(host or "")),
            "fetched_at": _now_iso(),
            "elapsed_ms": _elapsed_ms(t0),
            "fetch_ms": fetched_ms,
            "version": "d1-0.1",
        },
    )
    return response_json(result)

def rank_listings(urls: list[str], *, dry_run: bool = False) -> list[dict]:
    """Score a list of URLs and return rows sorted by absolute score desc."""
    rows: list[dict] = []
    sess = None if dry_run else (requests.Session() if requests else None)

    abs_scores: list[float] = []
    tmp: list[dict] = []
    for u in urls:
        r = score_url(u, dry_run=dry_run, session=sess)
        tmp.append(r)
        abs_scores.append(r["score"]["absolute"])

    for r in tmp:
        r["score"]["percentile"] = _percentile(r["score"]["absolute"], abs_scores)
        rows.append(r)

    rows.sort(key=lambda d: d["score"]["absolute"], reverse=True)
    return rows

def to_dataframe(rows: list[dict]):
    if pd is None:
        raise RuntimeError("pandas not installed")
    flat = []
    for r in rows:
        base = {
            "url": r["url"],
            "score_abs": r["score"]["absolute"],
            "score_pct": r["score"].get("percentile"),
            "status": r["status"],
            "host": r["meta"].get("host"),
            "is_ebay": r["meta"].get("is_ebay"),
        }
        for sig in r["signals"]:
            base[f"sig_{sig['name']}"] = sig["value"] * sig["weight"]
        flat.append(base)
    return pd.DataFrame(flat).sort_values("score_abs", ascending=False)


In [ ]:
# Cell 7 -

print("\n== Smoke tests (dry_run) ==")
tests = [
    "notaurl",  # invalid
    "http://example.com/article",  # non-eBay
    "https://www.ebay.com/itm/123",  # eBay
]
for u in tests:
    out = score_url(u, dry_run=True)
    print(u, "=>", json.dumps(out["score"], indent=2))

print("\n== Batch ranking (dry_run) ==")
ranked = rank_listings([tests[1], tests[2]], dry_run=True)
for row in ranked:
    print(row["url"], row["score"])

if pd is not None:
    df = to_dataframe(ranked)
    display(df.head())



== Smoke tests (dry_run) ==
notaurl => {
  "absolute": 0.0,
  "percentile": null
}
http://example.com/article => {
  "absolute": 7.82,
  "percentile": null
}
https://www.ebay.com/itm/123 => {
  "absolute": 39.94,
  "percentile": null
}

== Batch ranking (dry_run) ==
https://www.ebay.com/itm/123 {'absolute': 39.94, 'percentile': 100.0}
http://example.com/article {'absolute': 7.82, 'percentile': 50.0}


,url,score_abs,score_pct,status,host,is_ebay,sig_domain_prior,sig_https,sig_content_length,sig_citations_links,sig_author_block_hint,sig_sentiment,sig_seller_feedback_pct,sig_seller_feedback_count,sig_top_rated,sig_returns_policy,sig_card_specificity_terms,sig_year_set_hint,sig_image_count,sig_shipping_from
0,https://www.ebay.com/itm/123,39.94,100.0,ok,www.ebay.com,True,0.070,0.040,0.014,0.000,0.015,0.025,0.119712,0.08,0.06,0.046,0.0952,0.06,0.0375,0.021
1,http://example.com/article,7.82,50.0,ok,example.com,False,0.042,0.016,0.014,0.008,0.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Cell 8 -

# --- Feature extractor: turn "signals" into a fixed vector ---
FEATURE_ORDER = [
    "domain_prior","https","content_length","citations_links","author_block_hint",
    "seller_feedback_pct","seller_feedback_count","top_rated","returns_policy",
    "card_specificity_terms","year_set_hint","image_count","shipping_from","sentiment",
]

def signals_to_features(signals: list[dict]) -> dict[str, float]:
    feats = {f: 0.0 for f in FEATURE_ORDER}
    for s in signals:
        feats[s["name"]] = s["value"] * s["weight"]  # interpretable contribution
    return feats

def rows_to_matrix(rows: list[dict]):
    X = []
    for r in rows:
        feats = signals_to_features(r["signals"])
        X.append([feats[f] for f in FEATURE_ORDER])
    return np.array(X, dtype=float)

# --- Calibration model (logistic regression) ---
class Calibrator:
    def __init__(self):
        self.pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("lr", LogisticRegression(max_iter=200)),
        ])
        self.trained = False

    def fit(self, X: np.ndarray, y: np.ndarray):
        self.pipe.fit(X, y)
        self.trained = True
        return self

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        if not self.trained:
            raise RuntimeError("Calibrator not trained.")
        return self.pipe.predict_proba(X)[:, 1]

# --- Training data ---
# Replace this demo block with your real labels when ready.
have_sklearn = all(obj is not None for obj in (np, Pipeline, LogisticRegression, StandardScaler, train_test_split))
if have_sklearn:
    # Create a slightly larger demo set so both classes have >= 2 samples.
    demo_urls = [
        "https://www.ebay.com/itm/111",
        "https://www.ebay.com/itm/222",
        "https://www.ebay.com/itm/333",
        "http://example.com/article-a",
        "http://example.org/article-b",
        "http://example.net/article-c",
    ]
    demo_rows = rank_listings(demo_urls, dry_run=True)
    X = rows_to_matrix(demo_rows)

    # Demo labels (placeholder!): treat eBay rows as 1, non-eBay as 0
    y = np.array([1 if r["meta"]["is_ebay"] else 0 for r in demo_rows], dtype=int)

    # Check class counts
    counts = np.bincount(y, minlength=2)
    print("Class counts (demo):", {cls: int(c) for cls, c in enumerate(counts)})

    # Decide whether stratification is possible
    can_stratify = counts.min() >= 2 and len(y) >= 4
    stratify_arg = y if can_stratify else None
    if not can_stratify:
        print("Note: Not enough samples per class for stratified split; using non-stratified split.")

    # Train/validate split
    Xtr, Xte, ytr, yte = train_test_split(
        X, y, test_size=0.4, random_state=42, stratify=stratify_arg, shuffle=True
    )
    calib = Calibrator().fit(Xtr, ytr)
    yhat = calib.predict_proba(Xte)
    print("Calibration demo complete. Example predicted probs:", np.round(yhat, 3))
else:
    print("sklearn/numpy not available; skip calibration demo.")


Class counts (demo): {0: 3, 1: 3}
Calibration demo complete. Example predicted probs: [0.073 0.124 0.952]


In [ ]:
# Cell 9 -

def blend_score(heuristic_abs: float, calibrated_prob: float | None) -> float:
    """
    Blend heuristic 0..100 with calibrated prob 0..1 (if available).
    Simple linear mix (70% heuristic, 30% calibrated). Tweak as desired.
    """
    if calibrated_prob is None or not have_sklearn:
        return heuristic_abs
    return round(0.7 * heuristic_abs + 0.3 * (calibrated_prob * 100.0), 2)

def score_url_with_calibration(
    url: str,
    *,
    dry_run: bool = False,
    calibrator: Calibrator | None = None,
) -> dict:
    base = score_url(url, dry_run=dry_run)
    calibrated_prob = None
    if calibrator is not None and have_sklearn:
        feats = rows_to_matrix([base])  # uses base["signals"] internally
        # rows_to_matrix expects rows with ["signals"]; we wrap single base row
        # but rows_to_matrix currently expects a list of dicts with signals list only
        # So adapt minimally:
        feats = np.array([[signals_to_features(base["signals"])[f] for f in FEATURE_ORDER]], dtype=float)
        calibrated_prob = float(calibrator.predict_proba(feats)[0])

    blended = blend_score(base["score"]["absolute"], calibrated_prob)
    base["score"]["absolute"] = blended
    base["meta"]["calibrated_prob"] = calibrated_prob
    base["meta"]["version"] = "d2-0.1"  # bump version to show calibration applied
    return base

# Demo (only if we trained calib above)
if have_sklearn and 'calib' in globals() and isinstance(calib, Calibrator) and calib.trained:
    demo = score_url_with_calibration("https://www.ebay.com/itm/456", dry_run=True, calibrator=calib)
    print("Blended score:", demo["score"])


Blended score: {'absolute': 56.51, 'percentile': None}
